In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets bitsandbytes trl

In [ ]:
!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes

In [1]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset

2026-01-20 05:33:00.201210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768887180.239819     157 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768887180.247376     157 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768887180.265619     157 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768887180.265647     157 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768887180.265649     157 computation_placer.cc:177] computation placer alr

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True, 
                                         bnb_4bit_quant_type="nf4",  # "nf4" quantization type
                                         bnb_4bit_use_double_quant=True,  # Use double quantization
                                         bnb_4bit_compute_dtype=torch.float32  # Set compute dtype
                                        )

In [3]:
model_4bit = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map="auto",
    quantization_config=quantization_config, 
    dtype="auto"
)


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [19]:
print(model_4bit.get_memory_footprint()/1e6)

2673.000704


In [20]:
model_4bit

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [21]:
model_4bit = prepare_model_for_kbit_training(model_4bit)

In [22]:
config = LoraConfig(
    # the rank of the adapter, the lower the fewer parameters you'll need to train
    r=8,
    lora_alpha=16, # multiplier, usually 2*r
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # manually setting target modules
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj' 'gate_proj', 'up_proj', 'down_proj'],
    )

In [23]:
model_4bit = get_peft_model(model_4bit, config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [24]:
model_4bit

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): Qwen2ForCausalLM(
          (model): Qwen2Model(
            (embed_tokens): Embedding(151936, 2048)
            (layers): ModuleList(
              (0-35): 36 x Qwen2DecoderLayer(
                (self_attn): Qwen2Attention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [25]:
print(model_4bit.get_memory_footprint()/1e6)

2673.000704


In [26]:
train_p, tot_p = model_4bit.get_nb_trainable_parameters()
print(f'Trainable parameters: {train_p/1e6:.2f}M')
print(f'Total parameters: {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')

Trainable parameters: 10.03M
Total parameters: 3095.97M
% of trainable parameters: 0.32%


In [27]:
data_set = pd.read_parquet("/kaggle/input/telelogs-exp/telelogs_CoT.parquet")

In [28]:
data = data_set['q']
label = data_set['c']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=1)

In [30]:
train_data = pd.concat([X_train, y_train], axis = 1)
test_data = pd.concat([X_test, y_test], axis = 1)

In [31]:
train_data = train_data.reset_index()
test_data = test_data.reset_index()

In [32]:
train_data = train_data.drop("index", axis = 1)
test_data = test_data.drop("index", axis = 1)

In [33]:
train_data, test_data

(                                                      q   c
 0     Analyze the 5G wireless network drive-test use...  C6
 1     Analyze the 5G wireless network drive-test use...  C5
 2     Analyze the 5G wireless network drive-test use...  C2
 3     Analyze the 5G wireless network drive-test use...  C7
 4     Analyze the 5G wireless network drive-test use...  C3
 ...                                                 ...  ..
 1915  Analyze the 5G wireless network drive-test use...  C1
 1916  Analyze the 5G wireless network drive-test use...  C3
 1917  Analyze the 5G wireless network drive-test use...  C2
 1918  Analyze the 5G wireless network drive-test use...  C7
 1919  Analyze the 5G wireless network drive-test use...  C1
 
 [1920 rows x 2 columns],
                                                      q   c
 0    Analyze the 5G wireless network drive-test use...  C5
 1    Analyze the 5G wireless network drive-test use...  C3
 2    Analyze the 5G wireless network drive-test use...  C3


In [34]:
train_data.head(2), test_data.head(2)

(                                                   q   c
 0  Analyze the 5G wireless network drive-test use...  C6
 1  Analyze the 5G wireless network drive-test use...  C5,
                                                    q   c
 0  Analyze the 5G wireless network drive-test use...  C5
 1  Analyze the 5G wireless network drive-test use...  C3)

In [38]:
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

In [40]:
train_dataset, test_dataset

(Dataset({
     features: ['q', 'c'],
     num_rows: 1920
 }),
 Dataset({
     features: ['q', 'c'],
     num_rows: 480
 }))

In [41]:
train_dataset[0], test_dataset[0]

({'q': "Analyze the 5G wireless network drive-test user plane data and engineering parameters.\nIdentify the reason for the throughput dropping below 600Mbps in certain road sections.\nFrom the following 8 potential root causes, select the most likely one and enclose its number in \\boxed{{}} in the final answer.\n\nC1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.\nC2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.\nC3: A neighboring cell provides higher throughput.\nC4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.\nC5: Frequent handovers degrade performance.\nC6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.\nC7: Test vehicle speed exceeds 40km/h, impacting user throughput.\nC8: Average scheduled RBs are below 160, affecting throughput.\n\nGiven:\n- The default electronic downtilt value is 255, representing a downtilt angle of 6 degrees. Other 

In [42]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
tokenizer.chat_template

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- messages[0][\'content\'] }}\n    {%- else %}\n        {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}\n    {%- endif %}\n    {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}\n    {%- else %}\n       

In [50]:
messages = [
    {"role" : "user", "content" : train_dataset[0]['q']},
    {'role' : 'assistant', 'content' : train_dataset[0]['c']}
]
print(tokenizer.apply_chat_template(messages, tokenize=False))
messages

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the reason for the throughput dropping below 600Mbps in certain road sections.
From the following 8 potential root causes, select the most likely one and enclose its number in \boxed{{}} in the final answer.

C1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.
C2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.
C3: A neighboring cell provides higher throughput.
C4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.
C5: Frequent handovers degrade performance.
C6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.
C7: Test vehicle speed exceeds 40km/h, impacting user throughput.
C8: Average scheduled RBs are below 160, affecting throughput.

Given:
- The 

[{'role': 'user',
  'content': "Analyze the 5G wireless network drive-test user plane data and engineering parameters.\nIdentify the reason for the throughput dropping below 600Mbps in certain road sections.\nFrom the following 8 potential root causes, select the most likely one and enclose its number in \\boxed{{}} in the final answer.\n\nC1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.\nC2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.\nC3: A neighboring cell provides higher throughput.\nC4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.\nC5: Frequent handovers degrade performance.\nC6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.\nC7: Test vehicle speed exceeds 40km/h, impacting user throughput.\nC8: Average scheduled RBs are below 160, affecting throughput.\n\nGiven:\n- The default electronic downtilt value is 255, representing a downtilt an

In [86]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True, # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    # Actual batch (for updating) is same (1x) as micro-batch size
    gradient_accumulation_steps=1,
    # The initial (micro) batch size to start off with
    per_device_train_batch_size=4,
    # If batch size would cause OOM, halves its size until it works
    auto_find_batch_size=True,
    ## GROUP 2: Dataset-related
    max_length=512,
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,
    dataset_text_field='q',
    ## GROUP 3: These are typical training parameters
    num_train_epochs=2,
    learning_rate=3e-4,
    # Optimizer
    # 8-bit Adam optimizer - doesn't help much if you're using LoRA!
    optim='paged_adamw_8bit',
    ## GROUP 4: Logging parameters
    logging_steps=400,
    logging_dir='./logs',
    output_dir='./Qwen/Qwen2.5-3B-Instruct',
    report_to='none'
    )

In [87]:
trainer = SFTTrainer(
    model = model_4bit,
    processing_class = tokenizer,
    args = sft_config,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

Adding EOS to train dataset:   0%|          | 0/1920 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1920 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/1920 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

In [88]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [89]:
batch

{'input_ids': tensor([[16, 23, 20,  ..., 18, 18, 17]], device='cuda:0'),
 'labels': tensor([[-100,   23,   20,  ...,   18,   18,   17]], device='cuda:0'),
 'position_ids': tensor([[  0,   1,   2,  ..., 509, 510, 511]], device='cuda:0')}

In [ ]:
trainer.train()

Step,Training Loss
